In [34]:
import pandas
import numpy as np

In [35]:
All_probabilities = pandas.read_hdf('/home/amachlou/Documents/INRIA/Sulcal_Neuroanatomy/all_anterior_inferior_probabilities_RL_only_common_subjects.hdf')
All_probabilities = All_probabilities.rename({'inferior':'superior'})
All_probabilities

sulcus                                                      Lat_Fis-ant-Horizont  \
hemisphere sulcus                   probability_of Subject                         
left       Lat_Fis-ant-Horizont     anterior       102715                    NaN   
                                    superior       102715                    NaN   
           Lat_Fis-ant-Vertical     anterior       102715           7.816540e-06   
                                    superior       102715           9.966060e-01   
           Lat_Fis-post             anterior       102715           0.000000e+00   
                                    superior       102715           9.996392e-01   
           S_calcarine              anterior       102715           0.000000e+00   
                                    superior       102715           7.433579e-07   
           S_central                anterior       102715           0.000000e+00   
                                    superior       102715           1.000000e+00   
           S_cingul-Marginalis      anterior       102715           0.000000e+00   
                                    superior       102715           1.000000e+00   
           S_circular_insula_ant    anterior       102715           5.195230e-02   
                                    superior       102715           5.186942e-03   
           S_circular_insula_inf    anterior       102715           2.403413e-09   
                                    superior       102715           1.022933e-01   
           S_circular_insula_sup    anterior       102715           4.297712e-02   
                                    superior       102715           9.666300e-01   
           S_collat_transv_ant      anterior       102715           0.000000e+00   
                                    superior       102715           0.000000e+00   
           S_collat_transv_post     anterior       102715           0.000000e+00   
                                    superior       102715           0.000000e+00   
           S_front_inf              anterior       102715           2.972033e-01   
                                    superior       102715           9.999185e-01   
           S_front_middle           anterior       102715           8.453276e-01   
                                    superior       102715           9.999996e-01   
           S_front_sup              anterior       102715           2.211952e-01   
                                    superior       102715           1.000000e+00   
           S_interm_prim-Jensen     anterior       102715           0.000000e+00   
                                    superior       102715           1.000000e+00   
...                                                                          ...   
right      S_oc_sup_and_transversal anterior       994273           0.000000e+00   
                                    superior       994273           9.903859e-01   
           S_occipital_ant          anterior       994273           0.000000e+00   
                                    superior       994273           9.397115e-02   
           S_orbital-H_Shaped       anterior       994273           8.051229e-01   
                                    superior       994273           0.000000e+00   
           S_orbital_lateral        anterior       994273           9.999812e-01   
                                    superior       994273           4.105779e-01   
           S_orbital_med-olfact     anterior       994273           4.528448e-01   
                                    superior       994273           0.000000e+00   
           S_parieto_occipital      anterior       994273           0.000000e+00   
                                    superior       994273           9.999995e-01   
           S_pericallosal           anterior       994273           9.539183e-02   
                                    superior       994273           8.812661e-01   
           S_postcentral            anterior       9

In [36]:
hemispheres = list(All_probabilities.index.levels[0])

In [37]:
subject_ids = list(All_probabilities.index.levels[3])
subject_ids

[102715,
 102816,
 103111,
 103212,
 103414,
 103515,
 103818,
 104012,
 104416,
 104820,
 105014,
 105115,
 105216,
 105620,
 106016,
 106319,
 106521,
 107018,
 107321,
 107422,
 108121,
 108323,
 108525,
 108828,
 109123,
 109325,
 110411,
 111009,
 111312,
 111413,
 111716,
 112112,
 112819,
 113215,
 113619,
 113821,
 113922,
 114419,
 114924,
 115017,
 115320,
 116120,
 116524,
 117122,
 117324,
 117930,
 118124,
 118225,
 120717,
 121315,
 121416,
 121618,
 122317,
 122620,
 122822,
 123117,
 123420,
 123925,
 124220,
 124422,
 124826,
 125222,
 125424,
 125525,
 126931,
 127630,
 127832,
 127933,
 130619,
 130922,
 131217,
 131419,
 131621,
 131722,
 131823,
 131924,
 132118,
 133019,
 133625,
 133827,
 133928,
 134021,
 134324,
 134728,
 135124,
 135225,
 135528,
 135932,
 136227,
 136833,
 137027,
 137431,
 138534,
 139435,
 139839,
 140420,
 140925,
 141826,
 142424,
 143325,
 144226,
 145531,
 145632,
 146533,
 148133,
 148941,
 150524,
 150625,
 150726,
 151223,
 151425,
 

In [38]:
sulci_names = All_probabilities.columns
sulci_names

Index(['Lat_Fis-ant-Horizont', 'Lat_Fis-ant-Vertical', 'Lat_Fis-post',
       'S_calcarine', 'S_central', 'S_cingul-Marginalis',
       'S_circular_insula_ant', 'S_circular_insula_inf',
       'S_circular_insula_sup', 'S_collat_transv_ant', 'S_collat_transv_post',
       'S_front_inf', 'S_front_middle', 'S_front_sup', 'S_interm_prim-Jensen',
       'S_intrapariet_and_P_trans', 'S_oc-temp_lat',
       'S_oc-temp_med_and_Lingual', 'S_oc_middle_and_Lunatus',
       'S_oc_sup_and_transversal', 'S_occipital_ant', 'S_orbital-H_Shaped',
       'S_orbital_lateral', 'S_orbital_med-olfact', 'S_parieto_occipital',
       'S_pericallosal', 'S_postcentral', 'S_precentral-inf-part',
       'S_precentral-sup-part', 'S_suborbital', 'S_subparietal',
       'S_temporal_inf', 'S_temporal_sup', 'S_temporal_transverse'],
      dtype='object', name='sulcus')

In [39]:
def making_tables_of_each_sub_and_hem(all_probs, subject_ids, hemispheres):
    '''
    Input: pandas dataframe table of all subjects sulcal relations, both hemispheres
    Retrieves information of each subject's and each hemisphere's probabilities, and
    creates a dataframe table per subject per hemisphere
    '''
    subjects = {}
    
    for subject in subject_ids:
        hemisphere = {}
        for hem in hemispheres:
            sub_and_hem = all_probs.loc[(hem, slice(None), slice(None), subject)]
            hemisphere[hem]= sub_and_hem
        subjects[subject]=hemisphere
    
    return subjects
            
    

In [40]:
all_subs_and_hems = making_tables_of_each_sub_and_hem(All_probabilities, subject_ids, hemispheres)
all_subs_and_hems[102715]

{'left': sulcus                                   Lat_Fis-ant-Horizont  \
 sulcus                   probability_of                         
 Lat_Fis-ant-Horizont     anterior                         NaN   
                          superior                         NaN   
 Lat_Fis-ant-Vertical     anterior                7.816540e-06   
                          superior                9.966060e-01   
 Lat_Fis-post             anterior                0.000000e+00   
                          superior                9.996392e-01   
 S_calcarine              anterior                0.000000e+00   
                          superior                7.433579e-07   
 S_central                anterior                0.000000e+00   
                          superior                1.000000e+00   
 S_cingul-Marginalis      anterior                0.000000e+00   
                          superior                1.000000e+00   
 S_circular_insula_ant    anterior                5.195230e-02   
  

In [41]:
def making_sets_of_relative_sulci_per_hemisphere_and_subject(all_subs_and_hems, sulci_names):
    '''
    Creates a quadruple dictionary with keys: subject number, hemisphere, reference sulcus, relative position
    
    all_sulci_sets[subject][hemisphere][reference_sulcus][relative_position]
    
    Parameters: all_subs_and_hems: dictionary with keys: subject number, hemisphere
                sulci_names: pandas dataframe index array
                             names of sulci in one hemisphere
    Returns: all_sulci_sets: dict
    
    '''
    
    all_sulci_sets = {}
    
    for subject in all_subs_and_hems:
        
        hemisphere = {}
        
        for hem in all_subs_and_hems[subject]:
            
            sulci_sets = {}

            for sulcus_name in sulci_names:
               
                current_sub_and_hem = all_subs_and_hems[subject][hem]
            
                anterior_probability = current_sub_and_hem.xs('anterior', level='probability_of')
                superior_probability = current_sub_and_hem.xs('superior', level='probability_of')
                anterior_sulci=set(anterior_probability[sulcus_name][(anterior_probability>0.95)[sulcus_name]].index)
                posterior_sulci=set(anterior_probability[sulcus_name][(anterior_probability<0.05)[sulcus_name]].index)
                superior_sulci=set(superior_probability[sulcus_name][(superior_probability>0.95)[sulcus_name]].index)
                inferior_sulci=set(superior_probability[sulcus_name][(superior_probability<0.05)[sulcus_name]].index)
                position = {}
                position['anterior']=anterior_sulci
                position['posterior']=posterior_sulci
                position['superior']=superior_sulci
                position['inferior']=inferior_sulci
                sulci_sets[sulcus_name]=position
            
            hemisphere[hem]=sulci_sets
        
        all_sulci_sets[subject]=hemisphere
    
    return all_sulci_sets

In [42]:
All_subject_and_hemisphere_sets = making_sets_of_relative_sulci_per_hemisphere_and_subject(all_subs_and_hems, sulci_names)

### Making sulci formulae

In [45]:
for a in range(len(subject_ids)):
        subject = subject_ids[a]

In [46]:
formula_for_prcs = (All_subject_and_hemisphere_sets[subject]['left']["S_central"]['anterior']).intersection(
    All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Horizont"]['superior'].union(
        All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Vertical"]['superior'].union(
            All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-post"]['superior']
        )
    )
)

In [47]:
formula_for_sfs = ((All_subject_and_hemisphere_sets[subject]['left']["S_precentral-sup-part"]['anterior']).union(
    All_subject_and_hemisphere_sets[subject]['left']["S_precentral-inf-part"]['anterior'])).intersection(
    All_subject_and_hemisphere_sets[subject]['left']["S_front_inf"]['superior']
)

In [48]:
formula_for_calcarine = (All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_oc_middle_and_Lunatus']['anterior']).intersection(
        All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['posterior']).intersection(
            All_subject_and_hemisphere_sets[subject]['left']['S_occipital_ant']['superior']))

In [49]:
formula_for_pocs = ((All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['anterior']).intersection((
    All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-ant-Horizont']['superior'].union(
        All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-ant-Vertical']['superior'].union(
            All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['superior']))).intersection(
                All_subject_and_hemisphere_sets[subject]['left']['S_central']['posterior'])))

In [50]:
formula_for_ifs = All_subject_and_hemisphere_sets[subject]['left']['S_front_sup']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_precentral-sup-part']['anterior'])

In [51]:
formula_for_pericallosal = All_subject_and_hemisphere_sets[subject]['left']['S_central']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_cingul-Marginalis']['inferior']).intersection(
        All_subject_and_hemisphere_sets[subject]['left']['S_calcarine']['anterior']).intersection(
            All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-ant-Horizont']['posterior']).intersection(
                All_subject_and_hemisphere_sets[subject]['left']['S_circular_insula_inf']['superior'])

In [52]:
formula_for_los = All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['inferior']).intersection(
       All_subject_and_hemisphere_sets[subject]['left']['S_temporal_transverse']['posterior'])

In [53]:
formula_for_sos = All_subject_and_hemisphere_sets[subject]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_oc_middle_and_Lunatus']['superior'])

#### Calculating how to find sulci from formulae

In [57]:
#def sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets, sulcus_formula):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for a in range(len(subject_ids)):
        subject = subject_ids[a]
        sulcus_test = sulcus_formula #Replacing the sulcus formula with a variable representing the sulcus formula returns the same output for each subject

        sulci_found[subject]=sulcus_test

    return sulci_found

In [242]:
def sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for a in range(len(subject_ids)):
        subject = subject_ids[a]
        sulcus_test = All_subject_and_hemisphere_sets[subject]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_oc_middle_and_Lunatus']['superior'])

        sulci_found[subject]=sulcus_test

    return sulci_found

In [247]:
PrCS_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
SFS_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
Calcarine_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
PoCS_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
IFS_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
Pericallosal_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
LOS_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)
SOS_please_work = sulcus_prob_from_formula_left(subject_ids, All_subject_and_hemisphere_sets)

In [78]:
def sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for a in range(len(subject_ids)):
        subject = subject_ids[a]
        sulcus_test = All_subject_and_hemisphere_sets[subject]['right']['S_intrapariet_and_P_trans']['posterior'].intersection(
    All_subject_and_hemisphere_sets[subject]['right']['S_oc_middle_and_Lunatus']['superior'])
        
        sulci_found[subject]=sulcus_test
        
    return sulci_found

In [79]:
PrCS_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
SFS_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
Calcarine_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
PoCS_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
IFS_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
Pericallosal_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
LOS_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)
SOS_right = sulcus_prob_from_formula_right(subject_ids, All_subject_and_hemisphere_sets)

In [80]:
SOS_right

{102715: {'S_oc_sup_and_transversal'},
 102816: set(),
 103111: set(),
 103212: {'S_oc_sup_and_transversal'},
 103414: {'S_oc_sup_and_transversal'},
 103515: set(),
 103818: {'S_interm_prim-Jensen'},
 104012: set(),
 104416: {'S_oc_sup_and_transversal'},
 104820: set(),
 105014: set(),
 105115: {'S_oc_sup_and_transversal'},
 105216: set(),
 105620: {'S_interm_prim-Jensen', 'S_oc_sup_and_transversal'},
 106016: {'S_oc_sup_and_transversal'},
 106319: {'S_oc_sup_and_transversal'},
 106521: {'S_oc_sup_and_transversal'},
 107018: set(),
 107321: {'S_oc_sup_and_transversal'},
 107422: {'S_oc_sup_and_transversal'},
 108121: {'S_interm_prim-Jensen', 'S_oc_sup_and_transversal'},
 108323: set(),
 108525: set(),
 108828: set(),
 109123: set(),
 109325: {'S_oc_sup_and_transversal'},
 110411: set(),
 111009: set(),
 111312: set(),
 111413: {'S_oc_sup_and_transversal'},
 111716: set(),
 112112: {'S_oc_sup_and_transversal'},
 112819: set(),
 113215: set(),
 113619: {'S_oc_sup_and_transversal'},
 1138

### Calculating probability of sulcus found, out of all 572 subjects

In [97]:
both_PrCS = []

for a in range(0, 572):
    current_subject=(subject_ids[a])
    check_for_PrCS_limbs = 'S_precentral-sup-part' in PrCS_please_work[current_subject] and 'S_precentral-inf-part' in PrCS_please_work[current_subject]
    both_PrCS.append(check_for_PrCS_limbs)
        
prob_of_sulcus_found = (sum(both_PrCS))/(len(both_PrCS))
        
print(sum(both_PrCS))
print(len(both_PrCS))
print(prob_of_sulcus_found)

262
572
0.458041958041958


In [113]:
both_SFS = []

for a in range(0, 572):
    current_subject=(subject_ids[a])
    check_for_SFS = 'S_front_sup' in SFS_please_work[current_subject]
    both_SFS.append(check_for_SFS)
        
prob_of_sulcus_found = (sum(both_SFS))/(len(both_SFS))
        
print(sum(both_SFS))
print(len(both_SFS))
print(prob_of_sulcus_found)

225
572
0.39335664335664333


In [81]:
def calculating_prob_of_sulcus_found(subject_ids, sulci_output, desired_sulcus, destrieux_name):
    '''
    Input: subject_ids=list of subject ids, 
            sulci_output=dictionary, key: subject id, value: names of sulci returned from formula to find one sulcus
    Function iterates through sulci_output dictionary to check if desired sulcus is found in subject or not.
    Output: Total number of subjects for which desired sulcus was found
            Total number of subjects
            Probability of formula returning the desired sulcus within a subject.
    '''
    
    desired_sulcus = []

    for a in range(0, 572):
        current_subject=(subject_ids[a])
        check_for_sulcus = destrieux_name in sulci_output[current_subject]
        desired_sulcus.append(check_for_sulcus)

    prob_of_sulcus_found = (sum(desired_sulcus))/(len(desired_sulcus))

    return sum(desired_sulcus), len(desired_sulcus), prob_of_sulcus_found

In [17]:
def calculating_prob_of_PrCS_found(subject_ids, sulci_output, desired_sulcus):
    '''
    Input: subject_ids=list of subject ids, 
            sulci_output=dictionary, key: subject id, value: names of sulci returned from formula to find one sulcus
    Function iterates through sulci_output dictionary to check if desired sulcus is found in subject or not.
    Output: Total number of subjects for which desired sulcus was found
            Total number of subjects
            Probability of formula returning the desired sulcus within a subject.
    '''
    
    desired_sulcus = []

    for a in range(0, 572):
        current_subject=(subject_ids[a])
        check_for_sulcus = 'S_precentral-inf-part' in sulci_output[current_subject] and 'S_precentral-sup-part' in sulci_output[current_subject]
        desired_sulcus.append(check_for_sulcus)

    prob_of_sulcus_found = (sum(desired_sulcus))/(len(desired_sulcus))

    return sum(desired_sulcus), len(desired_sulcus), prob_of_sulcus_found

In [249]:
SFS_probability_left = calculating_prob_of_sulcus_found(subject_ids, SFS_please_work, 'Superior_frontal_sulcus', 'S_front_sup')
Calcarine_probability_left = calculating_prob_of_sulcus_found(subject_ids, Calcarine_please_work, 'Calcarine_sulcus', 'S_calcarine')
PoCS_probability_left = calculating_prob_of_sulcus_found(subject_ids, PoCS_please_work, 'Postcentral_sulcus', 'S_postcentral')
IFS_probability_left = calculating_prob_of_sulcus_found(subject_ids, IFS_please_work, 'Inferior_frontal_sulcus', 'S_front_inf')
Pericallosal_probability_left = calculating_prob_of_sulcus_found(subject_ids, Pericallosal_please_work, 'Pericallosal_sulcus', 'S_pericallosal')
LOS_probability_left = calculating_prob_of_sulcus_found(subject_ids, LOS_please_work, 'Lateral_occipital_sulcus', 'S_oc-temp_lat')
SOS_probability_left = calculating_prob_of_sulcus_found(subject_ids, SOS_please_work, 'Superior_occipital_sulcus', 'S_oc_sup_and_transversal')

In [82]:
PrCS_probability_right = calculating_prob_of_PrCS_found(subject_ids, PrCS_right, 'Precentral_sulcus')
SFS_probability_right = calculating_prob_of_sulcus_found(subject_ids, SFS_right, 'Superior_frontal_sulcus', 'S_front_sup')
Calcarine_probability_right = calculating_prob_of_sulcus_found(subject_ids, Calcarine_right, 'Calcarine_sulcus', 'S_calcarine')
PoCS_probability_right = calculating_prob_of_sulcus_found(subject_ids, PoCS_right, 'Postcentral_sulcus', 'S_postcentral')
IFS_probability_right = calculating_prob_of_sulcus_found(subject_ids, IFS_right, 'Inferior_frontal_sulcus', 'S_front_inf')
Pericallosal_probability_right = calculating_prob_of_sulcus_found(subject_ids, Pericallosal_right, 'Pericallosal_sulcus', 'S_pericallosal')
LOS_probability_right = calculating_prob_of_sulcus_found(subject_ids, LOS_right, 'Lateral_occipital_sulcus', 'S_oc-temp_lat')
SOS_probability_right = calculating_prob_of_sulcus_found(subject_ids, SOS_right, 'Superior_occipital_sulcus', 'S_oc_sup_and_transversal')

In [258]:
PrCS_probability_right

(245, 572, 0.42832167832167833)

In [250]:
SOS_probability_left

(147, 572, 0.256993006993007)

In [83]:
SOS_probability_right

(257, 572, 0.4493006993006993)

In [246]:
LOS_probability_left

(572, 572, 1.0)

In [328]:
LOS_probability_right

(572, 572, 1.0)

In [236]:
Pericallosal_probability_left

(15, 572, 0.026223776223776224)

In [323]:
Pericallosal_probability_right

(1, 572, 0.0017482517482517483)

In [221]:
IFS_probability_left

(572, 572, 1.0)

In [318]:
IFS_probability_right

(490, 572, 0.8566433566433567)

In [222]:
PoCS_probability_left

(0, 572, 0.0)

In [305]:
PoCS_probability_right

(340, 572, 0.5944055944055944)

In [229]:
SFS_probability_left

(225, 572, 0.39335664335664333)

In [273]:
SFS_probability_right

(112, 572, 0.1958041958041958)

In [204]:
Calcarine_probability_left

(0, 572, 0.0)

In [77]:
Calcarine_probability_right

(62, 572, 0.10839160839160839)

In [103]:
PrCS_stats = calculating_prob_of_sulcus_found(subject_ids, PrCS_please_work)
PrCS_stats

262
572
0.458041958041958


In [29]:
('S_precentral-sup-part' in PrCS_prob[104820] and 'S_precentral-inf-part' in PrCS_prob[104820])#, len(PrCS_prob[123925]), len(PrCS_prob[124220])

True

In [317]:
prcs_prob_all

({102715: ({'Lat_Fis-ant-Vertical',
    'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'},
   {'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'}),
  102816: ({'Lat_Fis-ant-Vertical',
    'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'},
   {'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'}),
  103111: ({'Lat_Fis-ant-Vertical',
    'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'},
   {'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'}),
  103212: ({'Lat_Fis-ant-Vertical',
    'S_front_inf',
    'S_front_middle',
    'S_front_sup',
    'S_precentral-inf-part',
    'S_precentral-sup-part'},
   {'S_front_inf',
 

In [89]:
S_oc_sup_and_transversal = (All_subject_and_hemisphere_sets[994273]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[994273]['left']['S_oc_middle_and_Lunatus']['superior'])).intersection(
                            All_subject_and_hemisphere_sets[103111]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[103111]['left']['S_oc_middle_and_Lunatus']['superior']))

S_oc_sup_and_transversal, 'S_oc_sup_and_transversal' in S_oc_sup_and_transversal, len(S_oc_sup_and_transversal)

({'S_oc_sup_and_transversal'}, True, 1)

In [216]:
S_oc_sup_and_transversal = (All_subject_and_hemisphere_sets[994273]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[994273]['left']['S_oc_middle_and_Lunatus']['superior'])), (All_subject_and_hemisphere_sets[103111]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[103111]['left']['S_oc_middle_and_Lunatus']['superior'])), (All_subject_and_hemisphere_sets[103212]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[103212]['left']['S_oc_middle_and_Lunatus']['superior'])), (All_subject_and_hemisphere_sets[102816]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[102816]['left']['S_oc_middle_and_Lunatus']['superior'])), (All_subject_and_hemisphere_sets[106016]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[106016]['left']['S_oc_middle_and_Lunatus']['superior']))

S_oc_sup_and_transversal, S_oc_sup_and_transversal.count("S_oc_sup_and_transversal"), len(S_oc_sup_and_transversal)

(({'S_oc_sup_and_transversal'},
  {'S_oc_sup_and_transversal'},
  {'S_oc_sup_and_transversal'},
  set(),
  set()),
 0,
 5)

In [86]:
All_subject_and_hemisphere_sets[103111]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(All_subject_and_hemisphere_sets[103111]['left']['S_oc_middle_and_Lunatus']['superior'])

{'S_oc_sup_and_transversal'}

In [2]:
def making_sulci_sets_for_each_subject(sulci_names, sub_and_hem):
    
    for i in subject_ids:
        right_hem_sulci_sets(sulci_names, right_sub_and_hem)
        left_hem_sulci_sets(sulci_names, left_sub_and_hem)

### Correspondence with dependency paths

In [47]:
'''
Precentral sulcus: 'S_precentral-sup-part','S_precentral-inf-par'
    Precentral sulcus is a sulcus (on the lateral surface)(in the frontal lobe) anterior to the central sulcus and
        superior to the ant.hor.LF or the ver.hor.LF or the post.LF
'''
    
(sulci_sets["S_central"]['anterior']).intersection(
    sulci_sets["Lat_Fis-ant-Horizont"]['superior'].union(
        sulci_sets["Lat_Fis-ant-Vertical"]['superior'].union(
            sulci_sets["Lat_Fis-post"]['superior']
        )
    )
)



{'Lat_Fis-ant-Vertical',
 'S_front_inf',
 'S_front_middle',
 'S_front_sup',
 'S_orbital_lateral',
 'S_precentral-inf-part',
 'S_precentral-sup-part'}

In [53]:
'''Superior frontal sulcus: 'S_front_sup'
    Superior frontal sulcus is a sulcus (on the lateral surface)(in the frontal lobe) anterior to the precentral sulcus (therefore the central sulcus) and
        superior to the ant.hor.LF or the ver.hor.LF or the post.LF
    '''

sfs_res = ((sulci_sets["S_precentral-sup-part"]['anterior']).union(
    sulci_sets["S_precentral-inf-part"]['anterior'])).intersection(
    sulci_sets["S_front_inf"]['superior']
)
            
print(sfs_res)
print('S_front_sup' in sfs_res, len(sfs_res))

{'S_front_sup'}
True 1


In [68]:
'''Calcarine sulcus: 'S_calcarine'
    Calcarine sulcus is a sulcus (on the medial surface)(in the occipital lobe) inferior to the parieto-occipital sulcus
'''

sulci_sets['S_parieto_occipital']['inferior'].intersection(
    sulci_sets['S_oc_middle_and_Lunatus']['anterior']).intersection(
        sulci_sets['Lat_Fis-post']['posterior']).intersection(
            sulci_sets['S_occipital_ant']['superior'])

{'S_calcarine', 'S_interm_prim-Jensen'}

In [72]:
'''Inferior temporal sulcus: 'S_temporal_inf'
    ITS is a sulcus (on the lateral surface)(in the temporal lobe) inferior to the superior temporal sulcus
    and superior to the pre-occipital notch (S_occipital_ant)
'''

(sulci_sets["S_temporal_sup"]['inferior']).intersection(
    sulci_sets['S_occipital_ant']['anterior'])

{'S_oc-temp_lat'}

In [89]:
'''Postcentral sulcus: 'S_postcentral'
    Postcentral sulcus is a sulcus (on the lateral surface)(in the parietal lobe) superior to the LF
    and anterior to the POS
'''

(sulci_sets['S_parieto_occipital']['anterior']).intersection((
    sulci_sets['Lat_Fis-ant-Horizont']['superior'].union(
        sulci_sets['Lat_Fis-ant-Vertical']['superior'].union(
            sulci_sets['Lat_Fis-post']['superior']))).intersection(
                sulci_sets['S_central']['posterior'])
    )


{'Lat_Fis-post',
 'S_interm_prim-Jensen',
 'S_intrapariet_and_P_trans',
 'S_postcentral'}

In [98]:
'''Inferior frontal sulcus: 'S_front_inf'
    Inferior frontal sulcus is a sulcus (on the lateral surfae) (in the frontal lobe) anterior to the precentral sulcus
    and inferior to the superior frontal sulcus
'''

sulci_sets['S_front_sup']['inferior'].intersection(
    sulci_sets['S_precentral-sup-part']['anterior'])

{'Lat_Fis-ant-Horizont',
 'Lat_Fis-ant-Vertical',
 'S_circular_insula_ant',
 'S_front_inf',
 'S_orbital-H_Shaped',
 'S_orbital_lateral',
 'S_orbital_med-olfact',
 'S_precentral-inf-part',
 'S_suborbital'}

In [115]:
'''Pericallosal sulcus: 'S_pericallosal'
    The (peri)callosal sulcus is a sulcus (on the medial surface) inferior to the central sulcus, cingulate sulcus and paracentral sulcus
    '''

sulci_sets['S_central']['inferior'].intersection(
    sulci_sets['S_cingul-Marginalis']['inferior']).intersection(
        sulci_sets['S_calcarine']['anterior']).intersection(
            sulci_sets['Lat_Fis-ant-Horizont']['posterior']).intersection(
                sulci_sets['S_circular_insula_inf']['superior'])

{'S_interm_prim-Jensen', 'S_pericallosal'}

In [125]:
'''Lateral occipit(o-temporal) sulcus: 'S_oc-temp_lat'
    LO(T)S is a sulcus (on the lateral surface)(on the border between the occipital and temporal lobes). 
    It is inferior to the parieto-occipital sulcus and the lateral fissure, and
    posterior to the lateral fissure
'''

sulci_sets['S_parieto_occipital']['inferior'].intersection(
    sulci_sets['Lat_Fis-post']['inferior']).intersection(
       sulci_sets['S_temporal_transverse']['posterior'])

{'S_calcarine',
 'S_collat_transv_post',
 'S_oc-temp_lat',
 'S_oc_middle_and_Lunatus',
 'S_occipital_ant'}

In [131]:
'''Superior occipital sulcus: 'S_oc_sup_and_transversal'
    The SOS is a sulcus (on the lateral surface)(in the occipital lobe) posterior to the intraparietal sulcus, 
    superior to the lunate sulcus
    '''

sulci_sets['S_intrapariet_and_P_trans']['posterior'].intersection(
    sulci_sets['S_oc_middle_and_Lunatus']['superior'])

{'S_oc_sup_and_transversal'}